<br><br>
# **MariaDB using By Pandas**
<br>
https://swalloow.github.io/db-to-dataframe<br>
**마리아DB**를 **Pandas**를 사용하여 Create, Read, Update, Delete 작업하기
1. 지금까지 Sqlite3를 사용했지만 용량이 넘치기 시작했다
1. 간단한 SQL 문법은 Sqlite3 로 익히더라고
1. 이제부터는 대부분의 내용들을 **MariaDB** 까지도 포함해서 저장 및 활용하기

In [6]:
# Psql 에 대해서 익히려고 했지만, odroid Xu4 등에서 설치가 안됨
# 범용적인 활용을 위해선 우선은 MariaDB를 익혀서 작업을 하고
# 추후에 AWS등을 활용하여 PostgreSQL를 설치가능하면 확장해 나아가기

# 아래의 3개 모듈을 활용한다
import pymysql       # MySQL cursor 연결
import pandas as pd  # 결과물 출력 및 생성

'1.2.6'

<br>
## **1 SQLalchemy 사용법**
SQLITE3 를 활용 https://www.haruair.com/blog/1682
1. DB별 다른 문법들을 일원화 해서 작업 가능하도록 돕는다
1. 한가지 DB를 익히고 아면 Connector만 변경시, 다른 DB에도 적용이 가능
1. Django 내부에서도 이를 활용하는 듯 하다

<br>
### **01 engine : DB 접속**
**Driver** &nbsp; :// &nbsp;  **filename** &nbsp;&nbsp; ex) sqlite:///:memory:

In [8]:
# Python 에서 DB를 쉽게 연결하도록 중계
import sqlalchemy    
sqlalchemy.__version__

'1.2.6'

In [16]:
# SQLITE3 에 접속하기
# echo는 로그를 위한 플래그로 순수 SQL 코드를 보여준다.
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

# SQLITE3 엔진을 실행 (실행시 최초 접속)
engine.execute("select 1").scalar()
# 현재는 생성된 테이블이 없다
engine.table_names() 

2018-06-21 11:44:23,214 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-06-21 11:44:23,215 INFO sqlalchemy.engine.base.Engine ()
2018-06-21 11:44:23,216 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-06-21 11:44:23,216 INFO sqlalchemy.engine.base.Engine ()
2018-06-21 11:44:23,217 INFO sqlalchemy.engine.base.Engine select 1
2018-06-21 11:44:23,218 INFO sqlalchemy.engine.base.Engine ()
2018-06-21 11:44:23,220 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-06-21 11:44:23,220 INFO sqlalchemy.engine.base.Engine ()


[]

2018-06-21 11:44:04,255 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-06-21 11:44:04,257 INFO sqlalchemy.engine.base.Engine ()


[]

<br>
### **02 Connection : 매핑선언**
SQLITE3 활용 https://www.haruair.com/blog/1682

In [13]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = 'users'

    id       = Column(Integer, primary_key=True)
    name     = Column(String)
    fullname = Column(String)
    password = Column(String)

    def __init__(self, name, fullname, password):
        self.name     = name
        self.fullname = fullname
        self.password = password

    def __repr__(self):
        return "<User('%s', '%s', '%s')>" % (self.name, self.fullname, self.password)